In [33]:
#import the dependencies

import pandas as pd

import gmaps
 
import requests

#import the api key

from config import g_key

In [34]:
# Store the CSV you saved created in part one into a DataFrame.

city_data_df = pd.read_csv("weather_data/cities.csv")

city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cidreira,BR,2022-05-22 05:36:19,-30.1811,-50.2056,55.27,81,97,4.07
1,1,Hithadhoo,MV,2022-05-22 05:28:08,-0.6000,73.0833,83.46,69,100,11.03
2,2,Norman Wells,CA,2022-05-22 05:35:11,65.2820,-126.8329,41.04,93,100,3.44
3,3,Hofn,IS,2022-05-22 05:36:19,64.2539,-15.2082,46.06,86,100,16.53
4,4,Marzuq,YE,2022-05-22 05:36:20,14.4000,46.4667,89.60,12,44,3.96


In [35]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [36]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [37]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [38]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [39]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [40]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [41]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Hithadhoo,MV,2022-05-22 05:28:08,-0.6000,73.0833,83.46,69,100,11.03
4,4,Marzuq,YE,2022-05-22 05:36:20,14.4000,46.4667,89.60,12,44,3.96
5,5,Puerto Ayora,EC,2022-05-22 05:28:05,-0.7393,-90.3518,71.08,89,56,5.82
8,8,San Patricio,US,2022-05-22 05:36:20,28.0170,-97.5169,82.29,93,43,5.99
11,11,Victoria,HK,2022-05-22 05:33:53,22.2855,114.1577,82.96,81,94,15.99
13,13,Kapaa,US,2022-05-22 05:28:05,22.0752,-159.3190,78.78,91,100,11.50
14,14,Albany,US,2022-05-22 05:29:13,42.6001,-73.9662,75.52,85,61,1.01
15,15,Kudahuvadhoo,MV,2022-05-22 05:36:21,2.6708,72.8944,83.30,76,91,13.20
16,16,Nioro,GM,2022-05-22 05:36:22,13.3500,-15.7500,72.66,75,91,5.84
17,17,Souillac,MU,2022-05-22 05:36:22,-20.5167,57.5167,77.38,69,75,9.22


In [42]:
preferred_cities_df.count()

City_ID       222
City          222
Country       222
Date          222
Lat           222
Lng           222
Max Temp      222
Humidity      222
Cloudiness    222
Wind Speed    222
dtype: int64

In [43]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hithadhoo,MV,83.46,-0.6000,73.0833,
4,Marzuq,YE,89.60,14.4000,46.4667,
5,Puerto Ayora,EC,71.08,-0.7393,-90.3518,
8,San Patricio,US,82.29,28.0170,-97.5169,
11,Victoria,HK,82.96,22.2855,114.1577,
13,Kapaa,US,78.78,22.0752,-159.3190,
14,Albany,US,75.52,42.6001,-73.9662,
15,Kudahuvadhoo,MV,83.30,2.6708,72.8944,
16,Nioro,GM,72.66,13.3500,-15.7500,
17,Souillac,MU,77.38,-20.5167,57.5167,


In [44]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [45]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
   # Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

    
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hithadhoo,MV,83.46,-0.6000,73.0833,
4,Marzuq,YE,89.60,14.4000,46.4667,
5,Puerto Ayora,EC,71.08,-0.7393,-90.3518,
8,San Patricio,US,82.29,28.0170,-97.5169,
11,Victoria,HK,82.96,22.2855,114.1577,
13,Kapaa,US,78.78,22.0752,-159.3190,
14,Albany,US,75.52,42.6001,-73.9662,
15,Kudahuvadhoo,MV,83.30,2.6708,72.8944,
16,Nioro,GM,72.66,13.3500,-15.7500,
17,Souillac,MU,77.38,-20.5167,57.5167,


In [46]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

AttributeError: module 'collections' has no attribute 'Iterable'

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [ ]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [1]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

NameError: name 'hotel_df' is not defined